In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [52]:
return_frame = pd.DataFrame([])

In [53]:

for filename in os.listdir("../data/RawData/BLS"):
    return_frame = pd.concat([return_frame,pd.read_csv("../data/RawData/BLS/"+filename).groupby("Year").agg(np.mean).rename(columns={"Value":filename.split("_")[0]})],axis=1)

In [2]:
def toQuarter(month):
    if month>=1 and month<=3:
        return 1
    elif month > 3 and month <=6:
        return 2
    elif month > 6 and month <= 9:
        return 3
    else: 
        return 4

In [15]:
def formatData(filename):
    data = pd.read_csv('../data/RawData/BLS/'+filename,usecols=["Year","Period","Value"])
    
    data.Period = data.Period.apply(lambda x: x[1:])
    data.Period = data.Period.astype("int")
    data["Quarter"] = data.Period.apply(toQuarter)
    data = data.drop(columns=["Period"])
    data = data.groupby(["Year","Quarter"]).mean()
    return data

In [35]:
training = pd.read_csv("../data/training.csv")

In [36]:

for filename in os.listdir("../data/RawData/BLS"):
    to_merge = formatData(filename).reset_index(level=[0,1])
    to_merge.rename(columns={"Value":filename.split("_")[0]},inplace=True)
    training = training.merge(to_merge,how="left",left_on=["YEAR FORECAST MADE","QUARTER"],right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
training.drop([col for col in training.columns if 'drop' in col], axis=1, inplace=True)
training.drop(columns = ["Year","Quarter"],inplace=True)

In [37]:
training

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,ACTUAL_CONF,HIT,pred_average,pred_var,chicken,coffee,eggs,electricity,rice,unleadedGasoline
0,1968,4,1968,NominalGNP,32,NaN,1968-32-1968-4,NominalGNP-1968-32-1968-4,0.0,0.0,...,0.00,0.0,6.650,0.627500,NaN,NaN,NaN,NaN,NaN,NaN
1,1968,4,1968,NominalGNP,77,NaN,1968-77-1968-4,NominalGNP-1968-77-1968-4,15.0,6.0,...,0.06,0.0,5.410,13.241900,NaN,NaN,NaN,NaN,NaN,NaN
2,1968,4,1968,NominalGNP,37,NaN,1968-37-1968-4,NominalGNP-1968-37-1968-4,0.0,1.0,...,0.01,0.0,2.890,4.357900,NaN,NaN,NaN,NaN,NaN,NaN
3,1968,4,1968,NominalGNP,53,NaN,1968-53-1968-4,NominalGNP-1968-53-1968-4,5.0,10.0,...,0.10,0.0,7.500,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,1968,4,1968,NominalGNP,66,NaN,1968-66-1968-4,NominalGNP-1968-66-1968-4,0.0,0.0,...,0.00,0.0,7.510,0.049900,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16212,2019,2,2022,Unemployment,587,2.0,2022-587-2019-2,Unemployment-2022-587-2019-2,0.0,0.0,...,0.98,1.0,4.400,0.502500,1.510667,4.306,1.342667,0.136667,0.776333,2.829333
16213,2019,2,2022,Unemployment,421,2.0,2022-421-2019-2,Unemployment-2022-421-2019-2,0.0,0.0,...,0.98,1.0,4.875,0.465625,1.510667,4.306,1.342667,0.136667,0.776333,2.829333
16214,2019,2,2022,Unemployment,548,2.0,2022-548-2019-2,Unemployment-2022-548-2019-2,0.0,0.0,...,0.98,1.0,4.200,0.622500,1.510667,4.306,1.342667,0.136667,0.776333,2.829333
16215,2019,2,2022,Unemployment,579,1.0,2022-579-2019-2,Unemployment-2022-579-2019-2,0.0,0.0,...,0.98,1.0,4.200,0.622500,1.510667,4.306,1.342667,0.136667,0.776333,2.829333


In [38]:
training.to_csv("../data/TrainingData/trainingWithItems.csv")